In [1]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    HumanMessage,
    SystemMessage,
)

# components for one episode
from scai.custom_chat_models.crfm import crfmChatLLM # model
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory

# meta prompt 
from scai.modules.meta_prompt.base import MetaPromptModel # model
from scai.modules.meta_prompt.prompts import META_PROMPTS # prompts

# user 
from scai.modules.user.base import UserModel # model
from scai.modules.user.prompts import USER_PROMPTS # prompts

# assistant
from scai.modules.assistant.base import AssistantModel # model
from scai.modules.assistant.prompts import ASSISTANT_PROMPTS # prompts
from scai.modules.assistant.models import AssistantPrompt # prompt

# task
from scai.modules.task.prompts import TASK_PROMPTS

In [2]:
# sample task
task_prompt = TASK_PROMPTS["task_prompt_1"]
# sample users 
user_prompt_1 = USER_PROMPTS["user_prompt_1"]
user_prompt_2 = USER_PROMPTS["user_prompt_2"]
# sample assistant
assistant_prompt = ASSISTANT_PROMPTS["assistant_prompt_1"]
# sample meta prompt
meta_prompt = META_PROMPTS["meta_prompt_1"]

In [26]:
# initialise buffer 
buffer = CustomConversationBufferWindowMemory(system_k=2, 
                                              chat_k=2, 
                                              user_k=2, 
                                              assistant_k=2,
                                              assistant_system_k=1,
                                              ) # how many turns are stored will be used

system_message = "You are a helpful AI assistant :)."
buffer.save_context(system={"content": system_message})

In [28]:
assistant_model = AssistantModel(llm='chat_llm')

assistant_response = assistant_model.run(assistant_prompt=assistant_prompt, 
                                         task_prompt=task_prompt,
                                         buffer=buffer)

print(assistant_response)

System: You are a helpful AI assistant :). Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: Provide a response using no more than 50 tokens.


In [29]:
buffer.save_context(assistant={"content": "AI should respect human rights universally, but also account for cultural contexts. Sensitivity and neutrality should guide responses, without endorsing discrimination or biases."})

In [30]:
user_model_1 = UserModel(llm='chat_llm')
user_response_1 = user_model_1.run(user_prompt=user_prompt_1, 
                                 task_prompt=task_prompt, 
                                 buffer=buffer)

print(user_response_1)

System: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: AI should respect human rights universally, but also account for cultural contexts. Sensitivity and neutrality should guide responses, without endorsing discrimination or biases.
Human: Provide a response including feedback using no more than 50 tokens.


In [31]:
user_model_2 = UserModel(llm='chat_llm')
user_response_2 = user_model_1.run(user_prompt=user_prompt_2, 
                                 task_prompt=task_prompt, 
                                 buffer=buffer)

print(user_response_2)

System: You're an internet troll. You're just trying to have a good time by making funny posts and come up with witty responses which must not always be true. Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: AI should respect human rights universally, but also account for cultural contexts. Sensitivity and neutrality should guide responses, without endorsing discrimination or biases.
Human: Provide a response including feedback using no more than 50 tokens.


In [34]:
# includ user 1 feedback
buffer.save_context(user={"content": "The statement is well-crafted, but lacks sources to support the assertions. Can we include references from relevant literature or guidelines?."})
# include user 2 feedback
buffer.save_context(user={"content": "So, AI should wear a global diplomat's hat, eh? A universal rights champion with local cultural sensitivity. Sounds like a tightrope walk in clown shoes!"})

In [35]:
assistant_model = AssistantModel(llm='chat_llm')

assistant_response = assistant_model.run(assistant_prompt=assistant_prompt, 
                                         task_prompt=task_prompt,
                                         buffer=buffer)

print(assistant_response)

System: You are a helpful AI assistant :). Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: The statement is well-crafted, but lacks sources to support the assertions. Can we include references from relevant literature or guidelines?.
Human: So, AI should wear a global diplomat's hat, eh? A universal rights champion with local cultural sensitivity. Sounds like a tightrope walk in clown shoes!
Human: The statement is well-crafted, but lacks sources to support the assertions. Can we include references from relevant literature or guidelines?.
Human: So, AI should wear a global diplomat's hat, eh? A universal rights champion with local cultural sensitivity. Sounds like a tightrop

In [37]:
meta_model = MetaPromptModel(llm='chat_llm')
meta_response = meta_model.run(meta_prompt=meta_prompt, 
                               buffer=buffer)
# print(meta_response)
print(buffer.chat_memory)

messages=[AIMessage(content='AI should respect human rights universally, but also account for cultural contexts. Sensitivity and neutrality should guide responses, without endorsing discrimination or biases.', additional_kwargs={}, example=False), HumanMessage(content='The statement is well-crafted, but lacks sources to support the assertions. Can we include references from relevant literature or guidelines?.', additional_kwargs={}, example=False), HumanMessage(content="So, AI should wear a global diplomat's hat, eh? A universal rights champion with local cultural sensitivity. Sounds like a tightrope walk in clown shoes!", additional_kwargs={}, example=False), HumanMessage(content='The statement is well-crafted, but lacks sources to support the assertions. Can we include references from relevant literature or guidelines?.', additional_kwargs={}, example=False), HumanMessage(content="So, AI should wear a global diplomat's hat, eh? A universal rights champion with local cultural sensitiv

In [ ]:
# # prompting a custom chat model
# from scai.custom_chat_models.crfm import crfmChatLLM

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

# chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
#                    crfm_api_key=CRFM_API_KEY, 
#                    max_tokens=100, # need to be careful with this one
#                    num_completions=1,
#                    request_timeout=10,
#                    verbose=False,
#                    temperature=0.9,
#                    )